In [11]:
import pandas as pd
import gzip
import os
import json
import scipy as scipy
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import numpy as np
from sklearn import  linear_model
from sklearn.model_selection import cross_val_predict

import multiprocessing

In [12]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
        return pd.DataFrame.from_dict(df, orient='index')
    
df = getDF('C:/Users/LeTrongTinh/Desktop/reviews_Apps_for_Android_5.json_2.gz')

In [13]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1N4O8VOJZTDVB,B004A9SDD8,Annette Yancey,"[1, 1]","Loves the song, so he really couldn't wait to ...",3.0,Really cute,1383350400,"11 2, 2013"


In [7]:
df.describe

<bound method NDFrame.describe of        reviewerID        asin    reviewerName helpful  \
0  A1N4O8VOJZTDVB  B004A9SDD8  Annette Yancey  [1, 1]   

                                          reviewText  overall      summary  \
0  Loves the song, so he really couldn't wait to ...      3.0  Really cute   

   unixReviewTime  reviewTime  
0      1383350400  11 2, 2013  >

In [13]:
df.shape

(1, 9)

In [8]:
import ipywidgets as widgets
from IPython.display import display
import statistics as stat

In [14]:
category_options={
    'Apps for Android': 'C:/Users/LeTrongTinh/Desktop/reviews_Apps_for_Android_5.json_2.gz',
}
dropdown = widgets.Dropdown(options=category_options)
display(dropdown)
if(not(os.path.isfile(dropdown.value[:-3]))):
    inputFile = gzip.open(dropdown.value, 'rb')
    outputFile = open(dropdown.value[:-3], 'wb')
    outputFile.write(inputFile.read())
    inputFile.close()
    outputFile.close()

In [35]:
data=[]
filepath=dropdown.value[:-3]
with open(filepath) as f:
    for line in f:
        data.append(json.loads(line))
df=pd.DataFrame(data)

In [118]:
print(df.shape)
df.head()

(752937, 9)


,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B004A9SDD8,"[1, 1]",3.0,"Loves the song, so he really couldn't wait to ...","11 2, 2013",A1N4O8VOJZTDVB,Annette Yancey,Really cute,1383350400
1,B004A9SDD8,"[0, 0]",5.0,"Oh, how my little grandson loves this app. He'...","12 5, 2011",A2HQWU6HUKIEC7,"Audiobook lover ""Kathy""",2-year-old loves it,1323043200
2,B004A9SDD8,"[0, 0]",5.0,I found this at a perfect time since my daught...,"05 21, 2012",A1SXASF6GYG96I,Barbara Gibbs,Fun game,1337558400
3,B004A9SDD8,"[3, 4]",5.0,My 1 year old goes back to this game over and ...,"12 6, 2012",A2B54P9ZDYH167,"Brooke Greenstreet ""Babylove""",We love our Monkeys!,1354752000
4,B004A9SDD8,"[1, 1]",5.0,There are three different versions of the song...,"02 1, 2014",AFOFZDTX5UC6D,C. Galindo,This is my granddaughters favorite app on my K...,1391212800


In [131]:
X_train=df.sample(frac=0.8,random_state=200)
X_test=df.drop(X_train.index)
print(X_train.shape, X_test.shape)

(602350, 9) (150587, 9)


In [138]:
y_train = X_train['overall']

In [132]:
X_train["reviewText"].astype(str)

293554    This is a fun game.  Too much dialogue. Have n...
351713    I hate this app because I play the cello for s...
413759    I like this version of Sims because it does no...
124132    The reason why this is so crappy because you h...
57527     This game is great.  Hours of fun and a unique...
647922    This game is easy to pick up just for a few mi...
708472    There is no challenge involved.  Only one who ...
15957     And here I was just talking about break out la...
291575    I have a 7 year old step daughter who has been...
224000    This app works better than the app on my lapto...
65229     I have this on both my phone and kindle. I rec...
344732    This game has a little of everything to offer....
462523    I got this while it was free for the day and i...
497915    This game is the best one so far for me,  espe...
533716    No real point to this app.  Needs to have a po...
281432    This app is a really great app for anyone that...
26380     I've tried to read The Bible b

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect_stopword = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')

In [134]:
corpus_train_reviewText=list(X_train["reviewText"].astype(str))
X_train_tfidf_reviewText = tfidf_vect_stopword.fit_transform(corpus_train_reviewText)
X_train_tfidf_reviewText.shape

(602350, 142654)

In [135]:
corpus_test_reviewText=list(X_test["reviewText"].astype(str))
X_test_tfidf_reviewText = tfidf_vect_stopword.fit_transform(corpus_test_reviewText)
X_test_tfidf_reviewText.shape

(150587, 61746)

In [140]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf_reviewText, y_train)
predicted_NaiveBayes_reviewText = clf.predict(X_test_tfidf_reviewText)

ValueError: dimension mismatch

In [58]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

In [84]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])


In [86]:
text_clf.fit(X_train.data, X_train.target)  
Pipeline('vect','tfidf','clf')

TypeError: __init__() takes 2 positional arguments but 4 were given

In [56]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In [ ]:
clf.coef_

In [61]:
clf.feature_log_prob_

array([[-10.50853956,  -9.62397992, -10.61118003, ..., -10.61118003,
        -10.61118003, -10.61118003],
       [ -9.79913451, -10.09898789, -10.60439042, ..., -10.60439042,
        -10.50404022, -10.60439042],
       [ -9.94462253, -10.04486966, -10.61983421, ..., -10.43681475,
        -10.63098399, -10.52134814],
       [ -9.43927044, -10.51505038, -10.64572491, ..., -10.64572491,
        -10.64572491, -10.64572491]])

In [64]:
predicted = clf.predict(X_test.data)
from sklearn import metrics
print(metrics.classification_report(X_test.target, predicted, target_names=X_test.target_names))

C:\Users\LeTrongTinh\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


TypeError: Cannot cast array data from dtype('float64') to dtype('<U32') according to the rule 'safe'